Actividad autónoma U2-T1 (AA3)

Brayan Cardenas




Objetivo de la actividad: Diseñar una solución orientada a objetos en 
Python que implemente desde cero el algoritmo K-Nearest Neighbors 
(KNN) para clasificar ejemplares del dataset penguins, analizando el 
efecto de distintos valores de k sobre la asignación de especie.

Instrucciones:  
Escenario: Clasificación de Pingüinos Antárticos        
Un instituto de biología necesita un clasificador que, a partir de 
mediciones morfológicas de pingüinos, prediga su especie (Adelie, 
Chinstrap o Gentoo).

In [4]:
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod

class KNNBase(ABC):
    """Clase abstracta que define los métodos esenciales de KNN."""

    @abstractmethod
    def fit(self, X, y):
        """Guarda los datos de entrenamiento."""
        pass

    @abstractmethod
    def predict(self, X_new, k=3):
        """Predice la clase de nuevos ejemplos usando KNN."""
        pass

    @abstractmethod
    def distance(self, p1, p2):
        """Calcula la distancia entre dos puntos."""
        pass

class KNNClassifier(KNNBase):
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict(self, X_new, k=3):
        X_new = np.array(X_new)
        predictions = []
        for x in X_new:
            distances = [self.distance(x, x_train) for x_train in self.X_train]
            idx_sorted = np.argsort(distances)
            k_nearest = self.y_train[idx_sorted[:k]]
            # Votación mayoritaria
            values, counts = np.unique(k_nearest, return_counts=True)
            predictions.append(values[np.argmax(counts)])
        return predictions

    def distance(self, p1, p2):
        # Distancia Euclidiana
        return np.sqrt(np.sum((p1 - p2) ** 2))

# Ejemplo de uso:
# Cargar datos (ajusta la ruta y columnas según tu dataset)
# df = pd.read_csv('penguins.csv')
# X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].values
# y = df['species'].values

# knn = KNNClassifier()
# knn.fit(X, y)
# predicciones = knn.predict([[40.1, 17.0, 185, 3700]], k=3)
# print(predicciones)